# Data Understanding and Feature Engineering

### Load Libraries
Load the libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
from datetime import datetime
from time import strftime, localtime

In [2]:
import warnings
warnings.filterwarnings("ignore")

### Loading data

In [3]:
metar = pd.read_csv('../data/METAR_all_airports.csv')
arrivals = pd.read_csv('../data/arrivals.csv',sep='\t')
departures = pd.read_csv('../data/departures.csv',sep='\t')

### METAR Dataset

In [4]:
metar.shape

(13477, 5)

In [5]:
metar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13477 entries, 0 to 13476
Data columns (total 5 columns):
AIRPORT_IATA     13477 non-null object
AIRPORT_ICAO     13477 non-null object
UTC DATE/TIME    13477 non-null object
METAR            13477 non-null object
DESCRIPTION      13477 non-null object
dtypes: object(5)
memory usage: 526.6+ KB


In [6]:
metar.head()

,AIRPORT_IATA,AIRPORT_ICAO,UTC DATE/TIME,METAR,DESCRIPTION
0,LHR,EGLL,2019-05-18 23:50:00,EGLL 182350Z AUTO 01003KT 9999 NCD 11/09 Q1009...,Day: 18th Time: 23:50 UTC Wind direction: 10 W...
1,LHR,EGLL,2019-05-18 23:20:00,EGLL 182320Z AUTO 02002KT 9999 NCD 10/08 Q1009...,Day: 18th Time: 23:20 UTC Wind direction: 20 W...
2,LHR,EGLL,2019-05-18 22:50:00,EGLL 182250Z AUTO 04002KT 9999 NCD 12/09 Q1009...,Day: 18th Time: 22:50 UTC Wind direction: 40 W...
3,LHR,EGLL,2019-05-18 22:20:00,EGLL 182220Z AUTO 06002KT 9999 NCD 12/09 Q1008,Day: 18th Time: 22:20 UTC Wind direction: 60 W...
4,LHR,EGLL,2019-05-18 21:50:00,EGLL 182150Z AUTO VRB02KT 9999 NCD 13/09 Q1008...,Day: 18th Time: 21:50 UTC Wind speed: 2kt Temp...


### Feature Engineering for METAR All Airports Dataset

In [7]:
metar['Date Time'] = metar['UTC DATE/TIME'].apply(lambda x : datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
metar['Hour'] = metar['Date Time'].apply(lambda x : x.hour)
metar['Date'] = metar['Date Time'].apply(lambda x : x.date())

In [8]:
metar.head()

,AIRPORT_IATA,AIRPORT_ICAO,UTC DATE/TIME,METAR,DESCRIPTION,Date Time,Hour,Date
0,LHR,EGLL,2019-05-18 23:50:00,EGLL 182350Z AUTO 01003KT 9999 NCD 11/09 Q1009...,Day: 18th Time: 23:50 UTC Wind direction: 10 W...,2019-05-18 23:50:00,23,2019-05-18
1,LHR,EGLL,2019-05-18 23:20:00,EGLL 182320Z AUTO 02002KT 9999 NCD 10/08 Q1009...,Day: 18th Time: 23:20 UTC Wind direction: 20 W...,2019-05-18 23:20:00,23,2019-05-18
2,LHR,EGLL,2019-05-18 22:50:00,EGLL 182250Z AUTO 04002KT 9999 NCD 12/09 Q1009...,Day: 18th Time: 22:50 UTC Wind direction: 40 W...,2019-05-18 22:50:00,22,2019-05-18
3,LHR,EGLL,2019-05-18 22:20:00,EGLL 182220Z AUTO 06002KT 9999 NCD 12/09 Q1008,Day: 18th Time: 22:20 UTC Wind direction: 60 W...,2019-05-18 22:20:00,22,2019-05-18
4,LHR,EGLL,2019-05-18 21:50:00,EGLL 182150Z AUTO VRB02KT 9999 NCD 13/09 Q1008...,Day: 18th Time: 21:50 UTC Wind speed: 2kt Temp...,2019-05-18 21:50:00,21,2019-05-18


In [9]:
metar['DESCRIPTION'][0]

'Day: 18th Time: 23:50 UTC Wind direction: 10 Wind speed: 3kt Temperature: 11C Dew point: 9C Pressure: 1009 hPa Visibility: 9999 m'

In [10]:
DESCRIPTION_decoded = {"Wind_direction":[], "Wind_speed":[], "Temperature":[], "Dew_point":[], "Pressure":[], "Visibility":[]} 
  
for description in metar['DESCRIPTION']:
    
    Wind_direction = float('nan')
    Wind_speed = float('nan')
    Temperature = float('nan')
    Dew_point = float('nan')
    Pressure = float('nan')
    Visibility = float('nan')
    
    # For Wind direction
    Wind_direction_field = re.search('Wind direction: [0-9]+', description)  
    if Wind_direction_field is not None: 
        Wind_direction = float(re.search('[0-9]+',Wind_direction_field.group(0)).group(0))
    DESCRIPTION_decoded["Wind_direction"].append(Wind_direction) 
      
    # For Wind speed
    Wind_speed_field = re.search('Wind speed: [0-9]+', description)  
    if Wind_speed_field is not None: 
        Wind_speed = float(re.search('[0-9]+',Wind_speed_field.group(0)).group(0))
    DESCRIPTION_decoded["Wind_speed"].append(Wind_speed) 
    
    # For Temperature
    Temperature_field = re.search('Temperature: [0-9]+', description)  
    if Temperature_field is not None: 
        Temperature = float(re.search('[0-9]+',Temperature_field.group(0)).group(0))
    DESCRIPTION_decoded["Temperature"].append(Temperature) 
    
    # For Dew point
    Dew_point_field = re.search('Dew point: [0-9]+', description)  
    if Dew_point_field is not None: 
        Dew_point = float(re.search('[0-9]+',Dew_point_field.group(0)).group(0))
    DESCRIPTION_decoded["Dew_point"].append(Dew_point) 
    
    # For Pressure
    Pressure_field = re.search('Pressure: [0-9]+', description)  
    if Pressure_field is not None: 
        Pressure = float(re.search('[0-9]+',Pressure_field.group(0)).group(0))
    DESCRIPTION_decoded["Pressure"].append(Pressure)
    
    # For Visibility
    Visibility_field = re.search('Visibility: [0-9]+', description)  
    if Visibility_field is not None: 
        Visibility = float(re.search('[0-9]+',Visibility_field.group(0)).group(0))
    DESCRIPTION_decoded["Visibility"].append(Visibility) 

In [11]:
DESCRIPTION_decoded_data = pd.DataFrame(DESCRIPTION_decoded) 
metar = metar.join(DESCRIPTION_decoded_data)
metar.head()

,AIRPORT_IATA,AIRPORT_ICAO,UTC DATE/TIME,METAR,DESCRIPTION,Date Time,Hour,Date,Wind_direction,Wind_speed,Temperature,Dew_point,Pressure,Visibility
0,LHR,EGLL,2019-05-18 23:50:00,EGLL 182350Z AUTO 01003KT 9999 NCD 11/09 Q1009...,Day: 18th Time: 23:50 UTC Wind direction: 10 W...,2019-05-18 23:50:00,23,2019-05-18,10.0,3.0,11.0,9.0,1009.0,9999.0
1,LHR,EGLL,2019-05-18 23:20:00,EGLL 182320Z AUTO 02002KT 9999 NCD 10/08 Q1009...,Day: 18th Time: 23:20 UTC Wind direction: 20 W...,2019-05-18 23:20:00,23,2019-05-18,20.0,2.0,10.0,8.0,1009.0,9999.0
2,LHR,EGLL,2019-05-18 22:50:00,EGLL 182250Z AUTO 04002KT 9999 NCD 12/09 Q1009...,Day: 18th Time: 22:50 UTC Wind direction: 40 W...,2019-05-18 22:50:00,22,2019-05-18,40.0,2.0,12.0,9.0,1009.0,9999.0
3,LHR,EGLL,2019-05-18 22:20:00,EGLL 182220Z AUTO 06002KT 9999 NCD 12/09 Q1008,Day: 18th Time: 22:20 UTC Wind direction: 60 W...,2019-05-18 22:20:00,22,2019-05-18,60.0,2.0,12.0,9.0,1008.0,9999.0
4,LHR,EGLL,2019-05-18 21:50:00,EGLL 182150Z AUTO VRB02KT 9999 NCD 13/09 Q1008...,Day: 18th Time: 21:50 UTC Wind speed: 2kt Temp...,2019-05-18 21:50:00,21,2019-05-18,NaN,2.0,13.0,9.0,1008.0,9999.0


In [12]:
metar.to_csv('../data/metar_engineered.csv')

### Arrival Dataset

In [13]:
arrivals.shape

(145495, 22)

In [14]:
arrivals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145495 entries, 0 to 145494
Data columns (total 22 columns):
mode                              145495 non-null object
flight_number                     145495 non-null object
callsign                          139035 non-null object
aircraft_model_code               145423 non-null object
aircraft_model_description        140870 non-null object
aircraft_registration             140942 non-null object
airline_name                      145495 non-null object
airline_iata                      145495 non-null object
airline_icao                      145495 non-null object
flight_origin_code_iata           145495 non-null object
flight_origin_code_icao           145398 non-null object
flight_origin_name                145495 non-null object
flight_origin_time_offset         145495 non-null int64
flight_destination_code_iata      145495 non-null object
flight_destination_code_icao      145495 non-null object
flight_destination_name           1

In [15]:
arrivals.head()

,mode,flight_number,callsign,aircraft_model_code,aircraft_model_description,aircraft_registration,airline_name,airline_iata,airline_icao,flight_origin_code_iata,...,flight_origin_time_offset,flight_destination_code_iata,flight_destination_code_icao,flight_destination_name,flight_destination_time_offset,flight_departure_scheduled,flight_departure_real,flight_arrival_scheduled,flight_arrival_real,flight_duaration
0,arrivals,JL41,JAL41,B788,Boeing 787-8 Dreamliner,JA837J,Japan Airlines,JL,JAL,HND,...,32400,LHR,EGLL,London Heathrow Airport,3600,1558284900,1.558287e+09,1558329900,1.558329e+09,41940.0
1,arrivals,SA234,SAA234,A333,Airbus A330-343,ZS-SXK,South African Airways,SA,SAA,JNB,...,7200,LHR,EGLL,London Heathrow Airport,3600,1558288800,1.558290e+09,1558329900,1.558329e+09,39720.0
2,arrivals,QF1,QFA1,A388,Airbus A380-842,VH-OQA,Qantas,QF,QFA,SIN,...,28800,LHR,EGLL,London Heathrow Airport,3600,1558281300,1.558286e+09,1558329900,1.558333e+09,46860.0
3,arrivals,BI3,RBA003,B788,Boeing 787-8 Dreamliner,V8-DLD,Royal Brunei Airlines,BI,RBA,BWN,...,28800,LHR,EGLL,London Heathrow Airport,3600,1558277700,1.558278e+09,1558329900,1.558329e+09,51120.0
4,arrivals,VS4,VIR4C,A333,Airbus A330-343,G-VWAG,Virgin Atlantic,VS,VIR,JFK,...,-14400,LHR,EGLL,London Heathrow Airport,3600,1558303200,1.558306e+09,1558329900,1.558329e+09,23220.0


### Departure Dataset

In [16]:
departures.shape

(282133, 22)

In [17]:
departures.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282133 entries, 0 to 282132
Data columns (total 22 columns):
mode                              282133 non-null object
flight_number                     282133 non-null object
callsign                          269817 non-null object
aircraft_model_code               281988 non-null object
aircraft_model_description        273030 non-null object
aircraft_registration             273175 non-null object
airline_name                      282133 non-null object
airline_iata                      282133 non-null object
airline_icao                      282133 non-null object
flight_origin_code_iata           282133 non-null object
flight_origin_code_icao           282036 non-null object
flight_origin_name                282133 non-null object
flight_origin_time_offset         282133 non-null int64
flight_destination_code_iata      282133 non-null object
flight_destination_code_icao      282022 non-null object
flight_destination_name           2

In [18]:
departures.head()

,mode,flight_number,callsign,aircraft_model_code,aircraft_model_description,aircraft_registration,airline_name,airline_iata,airline_icao,flight_origin_code_iata,...,flight_origin_time_offset,flight_destination_code_iata,flight_destination_code_icao,flight_destination_name,flight_destination_time_offset,flight_departure_scheduled,flight_departure_real,flight_arrival_scheduled,flight_arrival_real,flight_duaration
0,arrivals,JL41,JAL41,B788,Boeing 787-8 Dreamliner,JA837J,Japan Airlines,JL,JAL,HND,...,32400,LHR,EGLL,London Heathrow Airport,3600,1558284900,1.558287e+09,1558329900,1.558329e+09,41940.0
1,arrivals,SA234,SAA234,A333,Airbus A330-343,ZS-SXK,South African Airways,SA,SAA,JNB,...,7200,LHR,EGLL,London Heathrow Airport,3600,1558288800,1.558290e+09,1558329900,1.558329e+09,39720.0
2,arrivals,QF1,QFA1,A388,Airbus A380-842,VH-OQA,Qantas,QF,QFA,SIN,...,28800,LHR,EGLL,London Heathrow Airport,3600,1558281300,1.558286e+09,1558329900,1.558333e+09,46860.0
3,arrivals,BI3,RBA003,B788,Boeing 787-8 Dreamliner,V8-DLD,Royal Brunei Airlines,BI,RBA,BWN,...,28800,LHR,EGLL,London Heathrow Airport,3600,1558277700,1.558278e+09,1558329900,1.558329e+09,51120.0
4,arrivals,VS4,VIR4C,A333,Airbus A330-343,G-VWAG,Virgin Atlantic,VS,VIR,JFK,...,-14400,LHR,EGLL,London Heathrow Airport,3600,1558303200,1.558306e+09,1558329900,1.558329e+09,23220.0


### Feature Engineering for Arrival and Departure Dataset

In [19]:
arrivals['mode'].value_counts()

arrivals    145495
Name: mode, dtype: int64

In [20]:
departures['mode'].value_counts()

arrivals      145495
departures    136638
Name: mode, dtype: int64

In [21]:
departures_mode_arrivals = departures[departures['mode'] == 'arrivals']

In [22]:
merged_df = arrivals.merge(departures_mode_arrivals, indicator=True, how='outer')
changed_rows_df = merged_df[merged_df['_merge'] == 'right_only']
changed_rows_df.drop('_merge', axis=1)
len(changed_rows_df)

0

In [23]:
arrivals_departures = departures.drop_duplicates().reset_index()
arrivals_departures.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282133 entries, 0 to 282132
Data columns (total 23 columns):
index                             282133 non-null int64
mode                              282133 non-null object
flight_number                     282133 non-null object
callsign                          269817 non-null object
aircraft_model_code               281988 non-null object
aircraft_model_description        273030 non-null object
aircraft_registration             273175 non-null object
airline_name                      282133 non-null object
airline_iata                      282133 non-null object
airline_icao                      282133 non-null object
flight_origin_code_iata           282133 non-null object
flight_origin_code_icao           282036 non-null object
flight_origin_name                282133 non-null object
flight_origin_time_offset         282133 non-null int64
flight_destination_code_iata      282133 non-null object
flight_destination_code_icao      28

In [24]:
arrivals_departures['target'] = ''
for x in range(len(arrivals_departures)):
    if np.isnan(arrivals_departures['flight_arrival_real'][x]) and np.isnan(arrivals_departures['flight_departure_real'][x]):
        arrivals_departures['target'][x] = 'cancel'
    else:
        departure_delay = arrivals_departures['flight_departure_real'][x] - arrivals_departures['flight_departure_scheduled'][x]
        arrival_delay = arrivals_departures['flight_arrival_real'][x] - arrivals_departures['flight_arrival_scheduled'][x]
        if arrivals_departures['mode'][x] == 'arrivals' and arrival_delay > 0:
            arrivals_departures['target'][x] = 'delay'
        elif arrivals_departures['mode'][x] == 'departures' and departure_delay > 0:
            arrivals_departures['target'][x] = 'delay'
        else:
            arrivals_departures['target'][x] = 'neither delay nor cancel'

In [25]:
#arrivals_departures['departure_delay'] = arrivals_departures['flight_departure_real'] - arrivals_departures['flight_departure_scheduled']
#arrivals_departures['arrival_delay'] = arrivals_departures['flight_arrival_real'] - arrivals_departures['flight_arrival_scheduled']

In [26]:
arrivals_departures['Hour_departures'] = arrivals_departures['flight_departure_scheduled'].apply(lambda x : int(strftime('%H', localtime(x))))
arrivals_departures['Date_departures'] = arrivals_departures['flight_departure_scheduled'].apply(lambda x : strftime('%Y-%m-%d', localtime(x)))
arrivals_departures['Hour_arrivals'] = arrivals_departures['flight_arrival_scheduled'].apply(lambda x : int(strftime('%H', localtime(x))))
arrivals_departures['Date_arrivals'] = arrivals_departures['flight_arrival_scheduled'].apply(lambda x : strftime('%Y-%m-%d', localtime(x)))

In [27]:
arrivals_departures.to_csv('../data/arrivals_departures_engineered.csv')